In [ ]:
from datetime import datetime
from datetime import timedelta

from pymongo import MongoClient
from pandas import DataFrame
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import tushare as ts
from WindPy import w
w.start();


import fdm

In [ ]:
pro = ts.pro_api()

In [ ]:
client = MongoClient("192.168.56.1",27017)

In [ ]:
tushare = fdm.Tushare(client)
db = tushare.daily_basic()

In [ ]:
tushare.db.client

In [ ]:
MSCI_CN = {'CNY': '302400L.MI',
           'USD': '302400.MI'}
data = w.wsd("302400L.MI", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('msci_china.bz2')

In [ ]:
data = w.wsd("000300.SH", "close", "1999-12-20", "2019-08-26", "",usedf=True)[1]
data.sort_index(inplace = True)
data.to_json('hs300.bz2')

In [ ]:
index = pd.read_json('msci_china.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
index = pd.read_json('hs300.bz2').sort_index().dropna()
index_ror = index.pct_change()

In [ ]:
clean = fdm.CleanData(client)
cdb = clean.pricing()

tdb = fdm.TempDB(client)
actp = tdb['actPrice']

# Build active price vs CSI300
def actprice(code):
    start_date = datetime(2002,1,4)
    close = cdb.query({'code':code,'date':{'$gt':start_date}},['code','date','close'])
    if not close.empty:
        close.set_index(['date'],inplace = True)
        close = close.sort_index().copy()
        close['close'] = close['close'].pct_change()-index_ror['CLOSE']
        close.iat[0,1] = 0
        close['act_price'] = np.cumprod(close['close']+1)
        close.rename(columns={'close':'act_ror'},inplace=True)
        close.reset_index(inplace=True)
        return close
    else:
        return DataFrame

actp.drop()
codes = cdb.stock_codes()
for code in codes:
    df = actprice(code)
    actp.insert_many(df)



In [ ]:
actp.create_indexs(['date','code'])

In [ ]:
tdb = fdm.TempDB(client)
actp = tdb['actPrice']
mratio = tdb['mmRatio']

def maxmin_ratio(ts):
    maxv = np.max(ts)
    minv = np.min(ts)
    pt = ts[-1]
    return (minv-pt)/(minv-maxv)

#codes = actp.distinct('code')
mratio.drop()
for code in codes:
    data = actp.query({'code':code})
    data.set_index('date',inplace=True)
    data.sort_index(inplace=True)

    periods = [3,6,9,12,24,36]
    for n in periods:
        data['mm_ratio_'+str(n)] = data['act_price'].sort_index(ascending = False).rolling(20*n).apply(maxmin_ratio, raw=True)
    
    data.reset_index(inplace=True)
    mratio.insert_many(data)

In [ ]:
mratio.create_indexs(['date','code'])

In [ ]:
data = actp.query({'code':'000001.SZ'})
data.set_index('date',inplace=True)
data.sort_index(inplace=True)

In [ ]:
['CSI','SSE','SZSE','SW']
indexes = pro.index_basic(market='SW')
indexes.head()

# Factor base

In [ ]:
# Size factor

from fdm import Tushare
from fdm import CleanData

# Var
# client = self.get_client()
client = client
startdate = datetime(2019,1,1)
enddate = datetime(2019,2,1)
category = 'mv'
q=0.5

catcode = {'mv': 'total_mv', 'fmv': 'circ_mv'}

# Init database
tdb = Tushare(client).daily_basic()
cdb = CleanData(client).pricing()

# Get category score
catdata = tdb.query({'trade_date': {'$gt': startdate, '$lte': enddate}},
                    projection=['ts_code', 'trade_date', catcode[category]])
catdata = catdata.pivot(
    index='trade_date', columns='ts_code', values=catcode[category])
catdata.sort_index(inplace=True)

# Split stock codes by category score
def split_list_by_quantile(ts):
    low = ts[ts.quantile(q)>ts].index
    high = ts[ts.quantile(1-q)<=ts].index
    return pd.Series({'low':list(low),'high':list(high)})
split_list = catdata.apply(split_list_by_quantile,axis = 1)

# Get return data


In [ ]:
stocklist = split_list.at['2019-01-02','high']
cdb.query({'code':{'$in':stocklist},'date':datetime(2019,1,2)})#{'$gt': startdate, '$lte': enddate}

In [ ]:
DataFrame(cdb.col.col.find({'code':{'$in':stocklist},'date':{'$lte':datetime(2019,1,2)}}).sort([('date',-1)]).limit(2))

In [ ]:
df = cdb.query({'code':'000001.SZ'})
df = df.set_index('date')
df = df.sort_index()

In [ ]:

df['close']['2010-06-01':'2010-09-01'].plot(figsize=(20,5))

In [ ]:
df['close']['2010-06-01':'2010-09-01'].pct_change()

In [ ]:
datasource = fdm.CleanData(client).pricing()
tdb = fdm.TempDB(client)['tdb']
df = datasource.query({'date':{'$gte':datetime(2018,1,1)}})
tdb.insert_many(df)